<a href="https://colab.research.google.com/github/Lawrence-Krukrubo/SQL_for_Data_Science/blob/main/sql_for_data_analysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SQL AGGREGATIONS**

We connect to MySQL server and workbench and make analysis with the parch-and-posey database. This course is the practicals of the course **SQL for Data Analysis** at Udacity.

In [1]:
# Install mySQL connector

if False:
    !pip install mysql-connector-python

In [2]:
# we import some required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import time
print('Done!')

Done!


**Next, we create a connection to the parch-and-posey DataBase in MySQL Work-Bench**

In [3]:
import mysql
from mysql.connector import Error
from getpass import getpass

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='parch_and_posey',
                                         user=input('Enter UserName:'),
                                         password=getpass('Enter Password:'))
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

Enter UserName:danam
Enter Password:········
Connected to MySQL Server version  8.0.27
You're connected to database:  ('parch_and_posey',)


In [4]:
# Let's see the tables in parch-and-posey DB

# let's run the show tables command 

cursor.execute('show tables')
out = cursor.fetchall()
out

[('accounts',), ('orders',), ('region',), ('sales_reps',), ('web_events',)]

Let's see the first 3 data of the different tables in parch and posey database

Defining a method that converts a select query to a data frame

In [5]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 20) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [6]:
# 1. For the accounts table

query = 'SELECT * FROM accounts LIMIT 3;'
query_to_df(query)

Query ran for 0.20957684516906738 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510
2,1021,Apple,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520


In [7]:
# 2. For the orders table

query = 'SELECT * FROM orders LIMIT 3;'
query_to_df(query)

Query ran for 0.022989988327026367 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [8]:
# 3. For the sales_reps table

query = 'SELECT * FROM sales_reps LIMIT 3;'
query_to_df(query)

Query ran for 0.027982234954833984 secs!


,id,name,region_id
0,321500,Samuel Racine,1
1,321510,Eugena Esser,1
2,321520,Michel Averette,1


In [9]:
# 4. For the web_events table

query = 'SELECT * FROM web_events LIMIT 3;'
query_to_df(query)

Query ran for 0.015987157821655273 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


In [10]:
# 5. For the region table

query = 'SELECT * FROM region LIMIT 3;'
query_to_df(query)

Query ran for 0.011992931365966797 secs!


,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


**In essential, row-level data are useful for initial exploratory data analysis, when we're trying to get a feel of the data... But as we search for answers, aggregate-data which are often done along columns, become more useful...**

## Nulls:

NULLs are a datatype that specifies where no data exists in SQL. They are often ignored in our aggregation functions

* Notice that NULLs are different than a zero - they are cells where data does not exist.

* When identifying NULLs in a WHERE clause, we write IS NULL or IS NOT NULL. We don't use =, because NULL isn't considered a value in SQL. Rather, it is a property of the data.

**NULLs - Expert Tip**
* There are two common ways in which you are likely to encounter NULLs:

* NULLs frequently occur when performing a LEFT or RIGHT JOIN. You saw in the last lesson - when some rows in the left table of a left join are not matched with rows in the right table, those rows will contain some NULL values in the result set.

* NULLs can also occur from simply missing data in our database.

**COUNT the Number of Rows in each Table**

Try your hand at finding the number of rows in each table.

In [11]:
for table in ['orders','accounts','web_events','region','sales_reps']:
    query = f'SELECT COUNT(*) AS row_count FROM {table};'
    ans = query_to_df(query)
    print(f'Table {table}:')
    print(ans)
    print()

Query ran for 0.04397273063659668 secs!
Table orders:
   row_count
0       6912

Query ran for 0.003998994827270508 secs!
Table accounts:
   row_count
0        351

Query ran for 0.007994890213012695 secs!
Table web_events:
   row_count
0       9073

Query ran for 0.0039980411529541016 secs!
Table region:
   row_count
0          4

Query ran for 0.003995418548583984 secs!
Table sales_reps:
   row_count
0         50



### COUNT:

* Note that unlike other aggregations, `COUNT` can be used in columns of Non-Numerical values. Same too for `MIN` and `MAX` clauses.

* Notice that `COUNT` does not consider rows that have `NULL` values. Therefore, this can be useful for quickly identifying which rows have missing data. 

### SUM:

* Unlike `COUNT`, you can only use `SUM` on numeric columns. However, `SUM` will ignore NULL values, as do the other aggregation functions you will see in the upcoming lessons.

### Aggregation Reminder:

An important thing to remember: aggregators only aggregate vertically - the values of a column. If you want to perform a calculation across rows, you would do this with simple arithmetic.

### Aggregation Question

find the solution for each of the following questions. If you get stuck or want to check your answers, you can find the answers at the top of the next concept.

#### Q1: Find the total amount of poster_qty paper ordered in the orders table.

In [12]:
query = 'SELECT SUM(poster_qty) FROM orders;'
query_to_df(query)

Query ran for 0.010992288589477539 secs!


,SUM(poster_qty)
0,723646


#### Q2: Find the total amount of standard_qty paper ordered in the orders table.

In [13]:
query = 'SELECT SUM(standard_qty) FROM orders;'
query_to_df(query)

Query ran for 0.009989261627197266 secs!


,SUM(standard_qty)
0,1938346


#### Q4. Find the total dollar amount of sales using the total_amt_usd in the orders table.

In [14]:
query_to_df('SELECT SUM(total_amt_usd) FROM orders;')

Query ran for 0.0159912109375 secs!


,SUM(total_amt_usd)
0,23141511.83


#### Q5. Find the total amount spent on standard_amt_usd and gloss_amt_usd paper for each order in the orders table. This should give a dollar amount for each order in the table.

In [15]:
query_to_df(
    'SELECT id, (standard_amt_usd + gloss_amt_usd) tot_amt_usd FROM orders;'
)

Query ran for 9.673008918762207 secs!


,id,tot_amt_usd
0,1,778.55
1,2,1255.19
2,3,776.18
3,4,958.24
4,5,756.13
...,...,...
6907,6908,1545.40
6908,6909,706.54
6909,6910,783.90
6910,6911,816.20


#### Q6. Find the standard_amt_usd per unit of standard_qty paper. Your solution should use both an aggregation and a mathematical operator.

In [16]:
query_to_df(
 'SELECT (SUM(standard_amt_usd)  / SUM(standard_qty)) \
 standard_unit_usd FROM orders;'   
)

Query ran for 0.013989686965942383 secs!


,standard_unit_usd
0,4.990000


### Min and Max

Notice that `MIN` and `MAX` are aggregators that again ignore `NULL` values.

#### Expert Tip
Functionally, MIN and MAX are similar to COUNT in that they can be used on non-numerical columns. Depending on the column type, MIN will return the lowest number, earliest date, or non-numerical value as early in the alphabet as possible. As you might suspect, MAX does the opposite—it returns the highest number, the latest date, or the non-numerical value closest alphabetically to “Z.”

### AVG:

Similar to other software `AVG` returns the mean of the data - that is the sum of all of the values in the column divided by the number of values in a column. This aggregate function again ignores the `NULL` values in both the numerator and the denominator.

If you want to count NULLs as zero, you will need to use SUM and COUNT. However, this is probably not a good idea if the NULL values truly just represent unknown values for a cell.

#### MEDIAN - Expert Tip

One quick note that a median might be a more appropriate measure of center for this data, but finding the median happens to be a pretty difficult thing to get using SQL alone — so difficult that finding a median is occasionally asked as an interview question.

### Questions: MIN, MAX, & AVERAGE
Answer the following questions.

#### 1. When was the earliest order ever placed? You only need to return the date.

In [17]:
query_to_df(
    'SELECT MIN(occurred_at) earliest_order FROM orders;'
)

Query ran for 0.015995502471923828 secs!


,earliest_order
0,2013-12-04 04:22:44


#### 2. Try performing the same query as in question 1 without using an aggregation function.

In [18]:
query_to_df(
    'SELECT occurred_at earliest_order FROM orders ORDER BY earliest_order LIMIT 1;'
)

Query ran for 0.014990091323852539 secs!


,earliest_order
0,2013-12-04 04:22:44


#### 3. When did the most recent (latest) web_event occur?

In [19]:
query_to_df(
    'SELECT MAX(occurred_at) latest_event FROM web_events;'
)

Query ran for 0.01598811149597168 secs!


,latest_event
0,2017-01-01 23:51:09


#### 4. Try to perform the result of the previous query without using an aggregation function.

In [20]:
query_to_df(
    'SELECT occurred_at FROM web_events ORDER BY occurred_at DESC LIMIT 1;'
)

Query ran for 0.014996528625488281 secs!


,occurred_at
0,2017-01-01 23:51:09


#### 5. Find the mean (AVERAGE) amount spent per order on each paper type, as well as the mean amount of each paper type purchased per order. Your final answer should have 6 values - one for each paper type for the average number of sales, as well as the average amount.

In [21]:
query_to_df(
    "SELECT AVG(standard_amt_usd) avg_std_sum, \
    AVG(standard_qty) avg_std_qty, \
    AVG(gloss_amt_usd) avg_gloss_sum, \
    AVG(gloss_qty) avg_gloss_qty, \
    AVG(poster_amt_usd) avg_poster_sum, \
    AVG(poster_qty) avg_poster_qty \
    FROM orders;"
)

Query ran for 0.016994237899780273 secs!


,avg_std_sum,avg_std_qty,avg_gloss_sum,avg_gloss_qty,avg_poster_sum,avg_poster_qty
0,1399.355692,280.4320,1098.547420,146.6685,850.116539,104.6942


#### 6: Via the video, you might be interested in how to calculate the MEDIAN. Though this is more advanced than what we have covered so far try finding - what is the MEDIAN total_usd spent on all orders?

In [22]:
query_to_df(
    'SELECT * FROM \
    (SELECT total_amt_usd FROM orders ORDER BY total_amt_usd LIMIT 3457) \
    AS tot_amt ORDER BY total_amt_usd DESC LIMIT 2;'
)

Query ran for 0.015996694564819336 secs!


,total_amt_usd
0,2483.16
1,2482.55


In [53]:
query_to_df(
    "SELECT * FROM \
    (SET @row_index := -1; \
    SELECT @row_index:= @row_index + 1 AS row_index, total_amt_usd FROM orders ORDER BY total_amt_usd) \
    AS sub_query;"
)

DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now

## GROUP BY:

* `GROUP BY` can be used to aggregate data within subsets of the data. For example, grouping for different accounts, different regions, or different sales representatives.


* Any column in the `SELECT` statement that is not within an aggregator must be in the `GROUP BY` clause.


* The `GROUP BY` always goes between `WHERE` and `ORDER BY`.


* `ORDER BY` works like SORT in spreadsheet software.

### GROUP BY - Expert Tip:

SQL evaluates the aggregations before the `LIMIT` clause. If you don’t `group by` any columns, you’ll get a 1-row result—no problem there. If you `group by` a column with enough unique values that it exceeds the `LIMIT` number, the aggregates will be calculated, and then some rows will simply be omitted from the results.

This is actually a nice way to do things because you know you’re going to get the correct aggregates. If SQL cuts the table down to 100 rows, then performed the aggregations, your results would be substantially different. So the default style of `Group by` before `LIMIT` which usally comes last is ok.

## GROUP BY QUIZ:

Now that we've been introduced to `JOINs`, `GROUP BY`, and aggregate functions, the real power of SQL starts to come to life. Try some of the below to put your skills to the test!

One part that can be difficult to recognize is when it might be easiest to use an aggregate or one of the other SQL functionalities. Try some of the below to see if you can differentiate to find the easiest solution.

## Q1

Which account (by name) placed the earliest order? Your solution should have the account name and the date of the order.

In [23]:
query_to_df(
    'SELECT a.name acct_name, o.occurred_at date from accounts a JOIN \
    orders o ON a.id = o.account_id ORDER BY date LIMIT 1;'
)

Query ran for 0.023992300033569336 secs!


,acct_name,date
0,DISH Network,2013-12-04 04:22:44


## Q2

Find the total sales in usd for each account. You should include two columns - the total sales for each company's orders in usd and the company name.

In [24]:
query_to_df(
    'SELECT SUM(o.total_amt_usd) total_sales_usd, a.name acct_name FROM orders o \
    JOIN accounts a ON o.account_id = a.id GROUP BY acct_name;'
)

Query ran for 0.4337270259857178 secs!


,total_sales_usd,acct_name
0,124014.87,Walmart
1,2734.59,Exxon Mobil
2,20767.15,Apple
3,7474.32,Berkshire Hathaway
4,15184.46,McKesson
...,...,...
345,217473.85,KKR
346,2844.46,Oneok
347,9618.81,Newmont Mining
348,112176.03,PPL


## Q3

Via what channel did the most recent (latest) web_event occur, which account was associated with this web_event? Your query should return only three values - the date, channel, and account name.

In [25]:
query_to_df(
    'SELECT w.occurred_at date, w.channel channel, a.name acct_name FROM \
    web_events w JOIN accounts a ON w.account_id = a.id ORDER BY date DESC LIMIT 1;'
)

Query ran for 0.04597163200378418 secs!


,date,channel,acct_name
0,2017-01-01 23:51:09,organic,Molina Healthcare


## Q4

Find the total number of times each type of channel from the web_events was used. Your final table should have two columns - the channel and the number of times the channel was used.

In [26]:
query_to_df(
    'SELECT w.channel channel, COUNT(w.channel) count FROM web_events w GROUP BY \
    channel;'
)

Query ran for 0.027985572814941406 secs!


,channel,count
0,direct,5298
1,facebook,967
2,organic,952
3,adwords,906
4,twitter,474
5,banner,476


In [27]:
# Aggregating with DISTINCT...

query_to_df(
    'SELECT DISTINCT w.channel channel, COUNT(w.channel) count FROM web_events w \
    GROUP BY channel;'
)

Query ran for 0.03397631645202637 secs!


,channel,count
0,direct,5298
1,facebook,967
2,organic,952
3,adwords,906
4,twitter,474
5,banner,476


## Q5
Who was the primary contact associated with the earliest web_event?

In [28]:
query_to_df(
    'SELECT a.primary_poc FROM accounts a JOIN web_events w ON a.id = \
    w.account_id ORDER BY w.occurred_at LIMIT 1;'
)

Query ran for 0.016988039016723633 secs!


,primary_poc
0,Leana Hawker


## Q6

What was the smallest order placed by each account in terms of total usd. Provide only two columns - the account name and the total usd. Order from smallest dollar amounts to largest.


In [29]:
query_to_df(
    'SELECT a.name acct_name, MIN(o.total_amt_usd) min_order_usd FROM accounts \
     a JOIN orders o ON a.id = o.account_id GROUP BY acct_name ORDER BY \
     min_order_usd;'
)

Query ran for 0.3947618007659912 secs!


,acct_name,min_order_usd
0,Chevron,0.00
1,J.P. Morgan Chase,0.00
2,Boeing,0.00
3,Citigroup,0.00
4,Marathon Petroleum,0.00
...,...,...
345,Mohawk Industries,3025.65
346,Waste Management,3101.51
347,General Motors,4393.44
348,Berkshire Hathaway,7474.32


## Q7
Find the number of sales reps in each region. Your final table should have two columns - the region and the number of sales_reps. Order from fewest reps to most reps.

In [30]:
query_to_df(
    'SELECT r.name region, COUNT(s.name) sales_reps_count FROM region r JOIN \
    sales_reps s ON r.id = s.region_id GROUP BY region ORDER BY sales_reps_count;'
)

Query ran for 0.0099945068359375 secs!


,region,sales_reps_count
0,Midwest,9
1,Southeast,10
2,West,10
3,Northeast,21


I need to reconfirm the distinct channels in web_evnts again...

In [31]:
query_to_df(
    'SELECT DISTINCT(w.channel) distinct_channels FROM web_events w ORDER BY \
    distinct_channels;'
)

Query ran for 0.0569615364074707 secs!


,distinct_channels
0,adwords
1,banner
2,direct
3,facebook
4,organic
5,twitter


### **GROUP BY PART 2**

* We can `GROUP BY` multiple columns at once. This is often useful to aggregate across a number of different segments.

* The order of columns listed in the `ORDER BY` clause does make a difference. You are ordering the columns from left to right. But it makes no difference in `GROUP BY` Clause

**GROUP BY - Expert Tips**

* The order of column names in your `GROUP BY` clause doesn’t matter—the results will be the same regardless. If we run the same query and reverse the order in the `GROUP BY` clause, you can see we get the same results.


* As with `ORDER BY`, we can substitute numbers for column names in the `GROUP BY` clause. It’s generally recommended to do this only when you’re grouping many columns, or if something else is causing the text in the `GROUP BY` clause to be excessively long.


* A reminder here that any column that is not within an aggregation must show up in your `GROUP BY` statement. If you forget, you will likely get an error. However, in the off chance that your query does work, you might not like the results!

## GROUP BY Part II

### Q1
For each account, determine the average amount of each type of paper they purchased across their orders. Your result should have four columns - one for the account name and one for the average quantity purchased for each of the paper types for each account.

In [32]:
query_to_df(
    'SELECT a.name acct_name, AVG(o.standard_qty) ave_standard_qty, AVG(o.poster_qty) \
    ave_poster_qty, AVG(o.gloss_qty) ave_gloss_qty FROM accounts a JOIN orders o ON a.id \
    = o.account_id GROUP BY acct_name;'
)

Query ran for 0.49469447135925293 secs!


,acct_name,ave_standard_qty,ave_poster_qty,ave_gloss_qty
0,Walmart,282.0000,114.1786,279.6786
1,Exxon Mobil,527.0000,0.0000,14.0000
2,Apple,315.2000,17.5000,48.3000
3,Berkshire Hathaway,1148.0000,215.0000,0.0000
4,McKesson,167.2000,129.2000,154.0000
...,...,...,...,...
345,KKR,194.4909,155.3455,229.9273
346,Oneok,208.5000,23.5000,25.5000
347,Newmont Mining,90.0000,129.4000,56.6000
348,PPL,221.0690,127.3448,231.1034


### Q2
For each account, determine the average amount spent per order on each paper type. Your result should have four columns - one for the account name and one for the average amount spent on each paper type.

In [33]:
query_to_df(
    'SELECT a.name acct_name, AVG(o.standard_amt_usd) ave_standard_usd, AVG(o.poster_amt_usd) \
    ave_poster_usd, AVG(o.gloss_amt_usd) ave_gloss_usd FROM accounts a JOIN orders o ON a.id \
    = o.account_id GROUP BY acct_name;'
)

Query ran for 0.5086894035339355 secs!


,acct_name,ave_standard_usd,ave_poster_usd,ave_gloss_usd
0,Walmart,1407.180000,927.130000,2094.792500
1,Exxon Mobil,2629.730000,0.000000,104.860000
2,Apple,1572.848000,142.100000,361.767000
3,Berkshire Hathaway,5728.520000,1745.800000,0.000000
4,McKesson,834.328000,1049.104000,1153.460000
...,...,...,...,...
345,KKR,970.509636,1261.405091,1722.155273
346,Oneok,1040.415000,190.820000,190.995000
347,Newmont Mining,449.100000,1050.728000,423.934000
348,PPL,1103.134138,1034.040000,1730.964828


## Q3
Determine the number of times a particular channel was used in the web_events table for each sales rep. Your final table should have three columns - the name of the sales rep, the channel, and the number of occurrences. Order your table with the highest number of occurrences first.

In [34]:
query_to_df(
    'SELECT s.name sales_rep, w.channel channels, COUNT(w.channel) count FROM \
    sales_reps s JOIN accounts a ON s.id = a.sales_rep_id JOIN web_events w ON \
    w.account_id = a.id GROUP BY sales_rep, channels ORDER BY sales_rep, count DESC;'
)

Query ran for 0.4517178535461426 secs!


,sales_rep,channels,count
0,Akilah Drinkard,direct,53
1,Akilah Drinkard,organic,15
2,Akilah Drinkard,facebook,9
3,Akilah Drinkard,banner,9
4,Akilah Drinkard,adwords,5
...,...,...,...
290,Vernita Plump,adwords,43
291,Vernita Plump,organic,40
292,Vernita Plump,facebook,36
293,Vernita Plump,twitter,16


In [35]:
# Aggregating with DISTINCT

query_to_df(
    'SELECT DISTINCT s.name sales_rep, w.channel channels, COUNT(w.channel) count FROM \
    sales_reps s JOIN accounts a ON s.id = a.sales_rep_id JOIN web_events w ON \
    w.account_id = a.id GROUP BY sales_rep, channels ORDER BY sales_rep, count DESC;'
)

Query ran for 0.4657161235809326 secs!


,sales_rep,channels,count
0,Akilah Drinkard,direct,53
1,Akilah Drinkard,organic,15
2,Akilah Drinkard,facebook,9
3,Akilah Drinkard,banner,9
4,Akilah Drinkard,adwords,5
...,...,...,...
290,Vernita Plump,adwords,43
291,Vernita Plump,organic,40
292,Vernita Plump,facebook,36
293,Vernita Plump,twitter,16


### Q4
Determine the number of times a particular channel was used in the web_events table for each region. Your final table should have three columns - the region name, the channel, and the number of occurrences. Order your table with the highest number of occurrences first.


In [36]:
query_to_df(
    'SELECT r.name region, w.channel channels, COUNT(w.channel) count FROM \
    region r JOIN sales_reps s ON r.id = s.region_id JOIN accounts a ON s.id = \
    a.sales_rep_id JOIN web_events w ON w.account_id = a.id GROUP BY region, \
    channels ORDER BY region, count DESC;'
)

Query ran for 0.08295297622680664 secs!


,region,channels,count
0,Midwest,direct,696
1,Midwest,facebook,125
2,Midwest,organic,117
3,Midwest,adwords,101
4,Midwest,twitter,71
...,...,...,...
19,West,organic,243
20,West,adwords,241
21,West,facebook,229
22,West,twitter,122


### **Distinct**

* `DISTINCT` is always used in `SELECT` statements, and it provides the unique rows for all columns written in the `SELECT` statement. Therefore, you only use `DISTINCT` once in any particular `SELECT` statement.

* You could write:
```
SELECT DISTINCT column1, column2, column3
FROM table1;
```
which would return the unique (or DISTINCT) rows across all three columns.

* You could not write:
```
SELECT DISTINCT column1, DISTINCT column2, DISTINCT column3
FROM table1;
```
* You can think of DISTINCT the same way you might think of the statement "unique".


**DISTINCT - Expert Tip**

It’s worth noting that using `DISTINCT`, particularly in aggregations, can slow your queries down quite a bit.

## Q1 Distinct

Use DISTINCT to test if there are any accounts associated with more than one region.

In [37]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, COUNT(r.name) count FROM \
    accounts a JOIN sales_reps s ON a.sales_rep_id = s.id JOIN region r on \
    s.region_id = r.id GROUP BY acct_name ORDER BY count DESC;'
)

Query ran for 0.43173837661743164 secs!


,acct_name,count
0,Walmart,1
1,Exxon Mobil,1
2,Apple,1
3,Berkshire Hathaway,1
4,McKesson,1
...,...,...
346,KKR,1
347,Oneok,1
348,Newmont Mining,1
349,PPL,1


### Q2
Have any sales reps worked on more than one account? Answer using Distinct

In [38]:
query_to_df(
    'SELECT DISTINCT s.name sales_rep, COUNT(a.name) count \
    FROM sales_reps s JOIN accounts a on s.id = a.sales_rep_id GROUP BY sales_rep \
     ORDER BY count DESC;'
)

Query ran for 0.0819544792175293 secs!


,sales_rep,count
0,Georgianna Chisholm,15
1,Maren Musto,11
2,Dorotha Seawell,11
3,Earlie Schleusner,11
4,Micha Woodford,11
...,...,...
45,Retha Sears,3
46,Cordell Rieder,3
47,Ayesha Monica,3
48,Ernestine Pickron,3


## **Having**

**HAVING - Expert Tip**

HAVING is the “clean” way to filter a query that has been aggregated, but this is also commonly done using a subquery. Essentially, any time you want to perform a `WHERE` on an element of your query that was created by an aggregate, you need to use `HAVING` instead.

## **Pitching Where and Having**

1. `WHERE` subsets the returned data based on a logical condition
2. `WHERE` appears after the `FROM`, `JOIN` and `ON` clauses but before the `GROUP BY`
3. `HAVING` appears after the `GROUP BY` clause but before the `ORDER BY`.
4. `HAVING` is like `WHERE` but it works on logical statements involving aggregations.  

### Q

How many of the sales reps have more than 5 accounts that they manage?

In [39]:
query_to_df(
    'SELECT COUNT(*) num_reps FROM\
    (SELECT DISTINCT s.name sales_rep, COUNT(a.name) count FROM sales_reps s JOIN \
    accounts a on s.id = a.sales_rep_id GROUP BY sales_rep HAVING count > 5 \
    ORDER BY count) AS t1;'
)

Query ran for 0.009993791580200195 secs!


,num_reps
0,34


### Q

How many accounts have more than 20 orders?

In [40]:
query_to_df(
    'SELECT COUNT(*) num_accts FROM \
    (SELECT DISTINCT a.name acct_name, COUNT(o.account_id) orders FROM accounts a JOIN \
    orders o ON a.id = o.account_id GROUP BY acct_name HAVING orders > 20 \
    ORDER BY orders) AS t1;'
)

Query ran for 0.02599477767944336 secs!


,num_accts
0,120


### Q
Which account has the most orders?

In [41]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, COUNT(o.account_id) orders FROM accounts a \
    JOIN orders o ON a.id = o.account_id GROUP BY acct_name ORDER BY orders DESC\
    LIMIT 1;'
)

Query ran for 0.025990009307861328 secs!


,acct_name,orders
0,Leucadia National,71


### Q
How many accounts spent more than 30,000 usd total across all orders?

In [42]:
query_to_df(
    'SELECT COUNT(*) total_accts_over_30k FROM \
    (SELECT DISTINCT a.name acct_name, SUM(o.total_amt_usd) sum_total FROM accounts \
    a JOIN orders o on a.id=o.account_id GROUP BY acct_name HAVING sum_total > \
    30000 ORDER BY 2) AS t1;'
)

Query ran for 0.029979228973388672 secs!


,total_accts_over_30k
0,204


### Q
Which accounts spent less than 1,000 usd total across all orders?

In [43]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, SUM(o.total_amt_usd) total_spent FROM \
    accounts a JOIN orders o ON a.id=o.account_id GROUP BY acct_name HAVING \
    total_spent < 1000 ORDER BY total_spent DESC;'
)

Query ran for 0.03697562217712402 secs!


,acct_name,total_spent
0,Level 3 Communications,881.73
1,Delta Air Lines,859.64
2,Nike,390.25


### Q
Which account has spent the most with us?

In [44]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, SUM(o.total_amt_usd) max_total_spent FROM \
    accounts a JOIN orders o ON a.id=o.account_id GROUP BY acct_name ORDER BY \
    max_total_spent DESC LIMIT 1;'
)

Query ran for 0.04596972465515137 secs!


,acct_name,max_total_spent
0,EOG Resources,382873.30


### Q
Which account has spent the least with us?

In [45]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, SUM(o.total_amt_usd) min_total_spent FROM \
    accounts a JOIN orders o ON a.id=o.account_id GROUP BY acct_name ORDER BY \
    min_total_spent LIMIT 1;'
)

Query ran for 0.031984806060791016 secs!


,acct_name,min_total_spent
0,Nike,390.25


### Q
Which accounts used facebook as a channel to contact customers more than 6 times?

In [46]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, w.channel channels, COUNT(w.channel) count \
    FROM accounts a JOIN web_events w ON a.id=w.account_id WHERE w.channel LIKE \
    "%facebook%" GROUP BY acct_name, channels HAVING count > 6 ORDER BY count;'
)

Query ran for 0.07994937896728516 secs!


,acct_name,channels,count
0,Best Buy,facebook,7
1,J.P. Morgan Chase,facebook,7
2,Wells Fargo,facebook,7
3,Home Depot,facebook,7
4,Parker-Hannifin,facebook,7
...,...,...,...
41,Disney,facebook,12
42,Charter Communications,facebook,13
43,AutoNation,facebook,14
44,TJX,facebook,15


In [47]:
# Query can be written with only HAVING like so...

query_to_df(
    'SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel FROM accounts a \
    JOIN web_events w ON a.id = w.account_id GROUP BY a.id, a.name, w.channel \
    HAVING COUNT(*) > 6 AND w.channel LIKE "%facebook%" ORDER BY use_of_channel;'
)

Query ran for 0.18288779258728027 secs!


,id,name,channel,use_of_channel
0,1701,Best Buy,facebook,7
1,1221,J.P. Morgan Chase,facebook,7
2,1261,Wells Fargo,facebook,7
3,1271,Home Depot,facebook,7
4,3231,Parker-Hannifin,facebook,7
...,...,...,...,...
41,1521,Disney,facebook,12
42,3911,Charter Communications,facebook,13
43,2351,AutoNation,facebook,14
44,1881,TJX,facebook,15


### Q
Which account used facebook most as a channel?

In [48]:
query_to_df(
    'SELECT DISTINCT a.name acct_name, w.channel channels, COUNT(w.channel) count \
    FROM accounts a JOIN web_events w ON a.id=w.account_id WHERE w.channel LIKE \
    "%facebook%" GROUP BY 1, 2 ORDER BY 3 DESC LIMIT 1;'
)

Query ran for 0.027981996536254883 secs!


,acct_name,channels,count
0,Gilead Sciences,facebook,16


### Q
Which channel was most frequently used by most accounts?

In [49]:
query_to_df(
    'SELECT a.name acct_name, w.channel channels, COUNT(w.channel) count \
    FROM accounts a JOIN web_events w ON a.id=w.account_id GROUP BY acct_name, \
    channels ORDER BY count DESC LIMIT 10;'
)

Query ran for 0.0519709587097168 secs!


,acct_name,channels,count
0,Leucadia National,direct,52
1,Colgate-Palmolive,direct,51
2,New York Life Insurance,direct,51
3,Philip Morris International,direct,49
4,Charter Communications,direct,48
5,BlackRock,direct,48
6,ADP,direct,48
7,AutoNation,direct,48
8,FirstEnergy,direct,48
9,Western Digital,direct,47


In [50]:
# End the connection after running notebook

if not connection.is_connected():
    cursor.close()
    connection.close()
    print(f'Closing MySQL Connection to {record} Database')